**Import Libraries**

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import sys
import os
import padua
import matplotlib.pyplot as plt
import seaborn as sns
import platform
import scipy
import matplotlib.patches as mpatches

**Data Input**

In [ ]:
os.chdir(dir)
proteinGroups = padua.io.read_maxquant('proteinGroups.txt', low_memory = False)

**Select Columns**

In [51]:
# Save column-names to list
lscol = list(proteinGroups.columns)

# Create list with extracted LFQ-names
sub = ['LFQ']
colselect = []
collfq = []
for i in range(0,len(sub)):
    for text in lscol:
        if sub[i] in text:
            colselect.append(text)
            collfq.append(text)
 
# Extend list with additional column-names
ext = ['Only identified by site', 'Reverse', 'Potential contaminant','Protein IDs',
                  'Majority protein IDs', 'Peptide counts (all)',
                 'Number of proteins']

colselect.extend(ext)

# Select columns from this list
proteinGroups = proteinGroups[colselect]
old = collfq.copy()

In [ ]:
len(proteinGroups)

**Rename LFQ Columns**

In [53]:
colnew = collfq
for i in range(0,len(collfq)):
    colnew[i] = str(collfq[i].split('_')[3]).replace("F", "" )+'_'+str(collfq[i].split('_')[5])
new = colnew.copy()

In [54]:
colnew.sort()

In [55]:
gr1, gr2, gr3 = colnew[0:4], colnew[4:8], colnew[8:12]

In [56]:
for i in range(0,len(old)):
    proteinGroups = proteinGroups.rename(columns={old[i]: new[i]})

**Re-order**

In [57]:
proteinGroups = proteinGroups[gr1+gr2+gr3+ext]

**Filtering**

In [58]:
# Only proteins with at least 2 peptides identified with the first protein in the groups
for j in range(0,len(proteinGroups)):
    if int(proteinGroups['Peptide counts (all)'].loc[j].split(';')[0]) >= 2:
        pass
    else:
        proteinGroups = proteinGroups.drop([j])
proteinGroups = proteinGroups.reset_index(drop = True)

In [ ]:
len(proteinGroups)

In [60]:
# Exclude potential contaminant, reverse, only identified by site
proteinGroups = proteinGroups[proteinGroups['Potential contaminant']!='+']
proteinGroups = proteinGroups[proteinGroups['Only identified by site']!='+']
proteinGroups = proteinGroups[proteinGroups['Reverse']!='+']
proteinGroups = proteinGroups.reset_index(drop = True)

In [ ]:
len(proteinGroups)

In [62]:
# Only keep Proteins which have at least 50% valid values in at least one group
for j in range(0,len(proteinGroups)):
    x = 0
    y = 0
    z = 0
    for k in range(0,len(gr1)):
        if proteinGroups[gr1[k]].loc[j] != 0:
            x += 1
    for k in range(0,len(gr2)):
        if proteinGroups[gr2[k]].loc[j] != 0:
            y += 1
    for k in range(0,len(gr3)):
        if proteinGroups[gr3[k]].loc[j] != 0:
            z += 1
    if x >= len(gr1)/2 or y >= len(gr2)/2 or z >= len(gr3)/2:
        pass
    else:
        proteinGroups = proteinGroups.drop([j])
        
proteinGroups = proteinGroups.reset_index(drop = True)

In [ ]:
len(proteinGroups)

In [ ]:
# Ensure that at first position of column 'Majority protein IDs' is a HUMAN protein
# Replace with second place if necessary
# Exclude if no HUMAN protein at all available
for i in range(0,len(proteinGroups)):
    if 'HUMAN' in proteinGroups['Majority protein IDs'].loc[i]:
        if 'HUMAN' in proteinGroups['Majority protein IDs'].loc[i].split(';')[0]:
            pass
        else:
            xx = proteinGroups['Majority protein IDs'].loc[i].split(';')[1]
            if 'HUMAN' in xx:
                proteinGroups['Majority protein IDs'].loc[i] = xx
            else:
                print('ERROR: index ',i,' second place is not a human protein')     
    else:
        proteinGroups = proteinGroups.drop([i])
proteinGroups = proteinGroups.reset_index(drop = True)

In [ ]:
len(proteinGroups)

**Correct protein IDs**

In [66]:
n = []
for i in range(0,len(proteinGroups)):
    x = proteinGroups['Majority protein IDs'].loc[i]
    x = x.split(';')[0]
    n.append(x)
proteinGroups['Majority protein IDs'] = n

In [67]:
proteinGroups = proteinGroups.drop(columns = ['Only identified by site', 'Reverse', 'Potential contaminant', 'Protein IDs', 'Peptide counts (all)', 'Number of proteins'])

In [68]:
proteinGroups['UniProt ID'] = np.nan
proteinGroups['Entry Name'] = np.nan

for i in range(0,len(proteinGroups)):
    x = proteinGroups['Majority protein IDs'].loc[i].split('|')
    proteinGroups['UniProt ID'].loc[i] = x[1]
    proteinGroups['Entry Name'].loc[i] = x[2].split('_')[0]

**Extract number of measured values per condition**

In [69]:
proteinGroups['Control'] = np.nan
proteinGroups['Group_1'] = np.nan
proteinGroups['Group_2'] = np.nan

l1 = []
l2 = []
l3 = []

for i in range(0,len(proteinGroups)):
    o = proteinGroups.loc[i]
    x = 0
    y = 0
    z = 0
    for k in range(0,len(gr1)):
        if o[gr1[k]] != 0:
            x += 1
    for k in range(0,len(gr2)):
        if o[gr2[k]] != 0:
            y += 1        
    for k in range(0,len(gr3)):
        if o[gr3[k]] != 0:
            z += 1          
    l1.append(x)
    l2.append(y)
    l3.append(z)

proteinGroups['Control'] = l1
proteinGroups['Group_1'] = l2
proteinGroups['Group_2'] = l3

**Background**

In [70]:
bg = list(proteinGroups['UniProt ID'])

In [71]:
with open('Background.txt', 'w') as f:
    for line in bg:
        f.write(line)
        f.write('\n')

**Log2 Transformation**

In [72]:
for cols in gr1+gr2+gr3:
    proteinGroups[cols] = np.log2(proteinGroups[cols])

In [73]:
proteinGroups = proteinGroups.replace(np.inf, np.nan)
proteinGroups = proteinGroups.replace(-np.inf, np.nan)

In [74]:
proteinGroups.to_csv('Processing_Log2(X).csv')

In [ ]:
# Size
sns.set(style='whitegrid', rc={'figure.figsize':(15,10)})

# Iterate 
for columns in gr1+gr2+gr3:
    # Subset to the samples
    subset = proteinGroups[columns]
    
    # Draw the density plot
    sns.distplot(subset, hist = False, kde = True,
                 kde_kws = {'linewidth': 1},
                 label = columns)
    
# Plot formatting
plt.legend(prop={'size': 10}, title = 'Sample')
plt.xlabel('Log2(LFQ)')
plt.ylabel('Density')
plt.savefig('DensityPlot_Log2(LFQ).pdf')
plt.show()

In [ ]:
proteinGroups[gr1+gr2+gr3].isnull().sum(axis = 0)

**Mean Scaling**

In [ ]:
allcols = gr1+gr2+gr3
for c in allcols:
    proteinGroups[c] = proteinGroups[c] - np.mean(proteinGroups[c])

In [ ]:
# Size
sns.set(style='whitegrid', rc={'figure.figsize':(15,10)})

# Iterate 
for columns in gr1+gr2+gr3:
    # Subset to the samples
    subset = proteinGroups[columns]
    
    # Draw the density plot
    sns.distplot(subset, hist = False, kde = True,
                 kde_kws = {'linewidth': 1},
                 label = columns)
    
# Plot formatting
plt.legend(prop={'size': 10}, title = 'Sample')
plt.xlabel('Log2(LFQ)')
plt.ylabel('Density')
plt.savefig('Scaled_DensityPlot_Log2(X).pdf')
plt.show()

**Imputation**

In [79]:
gr1_imp = [x+'_imp' for x in gr1]
gr2_imp = [x+'_imp' for x in gr2]
gr3_imp = [x+'_imp' for x in gr3]
imp = gr1_imp+gr2_imp+gr3_imp

gr1_imp_1 = [x+'_imp_1' for x in gr1]
gr2_imp_1 = [x+'_imp_1' for x in gr2]
gr3_imp_1 = [x+'_imp_1' for x in gr3]
imp_1 = gr1_imp_1+gr2_imp_1+gr3_imp_1

gr1_imp_2 = [x+'_imp_2' for x in gr1]
gr2_imp_2 = [x+'_imp_2' for x in gr2]
gr3_imp_2 = [x+'_imp_2' for x in gr3]
imp_2 = gr1_imp_2+gr2_imp_2+gr3_imp_2

In [80]:
for i in range(0,len(imp)):
    proteinGroups[imp[i]] = np.nan
    proteinGroups[imp_1[i]] = proteinGroups[colnew[i]]
    proteinGroups[imp_2[i]] = proteinGroups[colnew[i]]

In [ ]:
# Determine how to impute
for i in range(0,len(proteinGroups)):
    x = proteinGroups['Control'].loc[i]
    y = proteinGroups['Group_1'].loc[i]
    z = proteinGroups['Group_2'].loc[i]
    
    
    if x == 4:
        for j in range(0,len(gr1)):
            proteinGroups[gr1_imp[j]].loc[i] = 0
    else:
        if x == 0 or x == 1:
            for j in range(0,len(gr1)):
                if pd.isna(proteinGroups[gr1[j]].loc[i]) == True:
                    proteinGroups[gr1_imp[j]].loc[i] = 2
                else:
                    proteinGroups[gr1_imp[j]].loc[i] = 0
        if x == 2 or x == 3:
            for j in range(0,len(gr1)):
                if pd.isna(proteinGroups[gr1[j]].loc[i]) == True:
                    proteinGroups[gr1_imp[j]].loc[i] = 1
                else:
                    proteinGroups[gr1_imp[j]].loc[i] = 0
                    
    if y == 4:
        for j in range(0,len(gr2)):
            proteinGroups[gr2_imp[j]].loc[i] = 0
    else:
        if y == 0 or y == 1:
            for j in range(0,len(gr2)):
                if pd.isna(proteinGroups[gr2[j]].loc[i]) == True:
                    proteinGroups[gr2_imp[j]].loc[i] = 2
                else:
                    proteinGroups[gr2_imp[j]].loc[i] = 0
        if y == 2 or y == 3:
            for j in range(0,len(gr2)):
                if pd.isna(proteinGroups[gr2[j]].loc[i]) == True:
                    proteinGroups[gr2_imp[j]].loc[i] = 1
                else:
                    proteinGroups[gr2_imp[j]].loc[i] = 0
                    
    if z == 4:
        for j in range(0,len(gr3)):
            proteinGroups[gr3_imp[j]].loc[i] = 0
    else:
        if z == 0 or z == 1:
            for j in range(0,len(gr3)):
                if pd.isna(proteinGroups[gr3[j]].loc[i]) == True:
                    proteinGroups[gr3_imp[j]].loc[i] = 2
                else:
                    proteinGroups[gr3_imp[j]].loc[i] = 0
        if z == 2 or z == 3:
            for j in range(0,len(gr3)):
                if pd.isna(proteinGroups[gr3[j]].loc[i]) == True:
                    proteinGroups[gr3_imp[j]].loc[i] = 1
                else:
                    proteinGroups[gr3_imp[j]].loc[i] = 0

In [82]:
df2 = proteinGroups[imp_2] 
df2 = padua.imputation.gaussian(df2[imp_2], width=0.3, downshift=-1.8, prefix=None)[0]

df = proteinGroups[imp_1] 
df = padua.imputation.gaussian(df[imp_1], width=0.3, downshift=-0.5, prefix=None)[0]

In [83]:
for i in range(0,len(proteinGroups)):
    for j in range(0,len(colnew)):
        x = proteinGroups[imp[j]].loc[i]
        if x == 0:
            pass
        else:
            if x == 1:
                proteinGroups[colnew[j]].loc[i] = df[imp_1[j]].loc[i]
            if x == 2:
                proteinGroups[colnew[j]].loc[i] = df2[imp_2[j]].loc[i]

In [ ]:
# Size
sns.set(style='whitegrid', rc={'figure.figsize':(15,10)})

# Iterate 
for columns in gr1+gr2+gr3:
    # Subset to the samples
    subset = proteinGroups[columns]
    
    # Draw the density plot
    sns.distplot(subset, hist = False, kde = True,
                 kde_kws = {'linewidth': 1},
                 label = columns)
    
# Plot formatting
plt.legend(prop={'size': 10}, title = 'Sample')
plt.xlabel('Log2(LFQ)')
plt.ylabel('Density')
plt.savefig('Imputed_DensityPlot_Log2(LFQ).pdf')
plt.show()

In [ ]:
pos1 = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]
pos2 = [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]

# Size
fig, axs = plt.subplots(3, 4, figsize=(10,10))

for k in range(0, len(imp)):
    axs[pos1[k], pos2[k]].hist(proteinGroups[proteinGroups[imp[k]]==0][colnew[k]], histtype='stepfilled', bins='auto', color='blue')
    axs[pos1[k], pos2[k]].hist(proteinGroups[proteinGroups[imp[k]]==1][colnew[k]], histtype='stepfilled', bins='auto', color='orange', alpha=0.75)
    axs[pos1[k], pos2[k]].hist(proteinGroups[proteinGroups[imp[k]]==2][colnew[k]], histtype='stepfilled', bins='auto', color='red', alpha=0.75)
    axs[pos1[k], pos2[k]].set_title(str(colnew[k]), size=12)
    
    
# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.grid(False)
    ax.label_outer()

plt.setp(axs[-1, 0:1], xlabel='Log2(LFQ)')
plt.setp(axs[2:3, 0], ylabel='Counts')
plt.savefig('Imputed_Histogram.pdf')
plt.show()

In [87]:
proteinGroups.to_csv('Processing_Imputation.csv')

In [88]:
proteinGroups = proteinGroups[allcols + ['Majority protein IDs', 'UniProt ID', 'Entry Name', 'Control', 'Group_1', 'Group_2']]

**Log2 Fold Change**

In [ ]:
proteinGroups['M1_vs_M2a_log2FC'] = np.nan
proteinGroups['M1_vs_M2c_log2FC'] = np.nan
proteinGroups['M2a_vs_M2c_log2FC'] = np.nan

In [ ]:
for i in range(0,len(proteinGroups)):
    zero = []
    one = []
    for j in range(0,len(gr1)):
        zero.append(proteinGroups[gr1[j]].loc[i])
    for j in range(0,len(gr2)):
        one.append(proteinGroups[gr2[j]].loc[i])
    zero = np.mean(zero)
    one = np.mean(one)
    lfc = zero - one
    proteinGroups['M1_vs_M2a_log2FC'].loc[i] = lfc
    
for i in range(0,len(proteinGroups)):
    zero = []
    one = []
    for j in range(0,len(gr1)):
        zero.append(proteinGroups[gr1[j]].loc[i])
    for j in range(0,len(gr3)):
        one.append(proteinGroups[gr3[j]].loc[i])
    zero = np.mean(zero)
    one = np.mean(one)
    lfc = zero - one
    proteinGroups['M1_vs_M2c_log2FC'].loc[i] = lfc

for i in range(0,len(proteinGroups)):
    zero = []
    one = []
    for j in range(0,len(gr2)):
        zero.append(proteinGroups[gr2[j]].loc[i])
    for j in range(0,len(gr3)):
        one.append(proteinGroups[gr3[j]].loc[i])
    zero = np.mean(zero)
    one = np.mean(one)
    lfc = zero - one
    proteinGroups['M2a_vs_M2c_log2FC'].loc[i] = lfc
    
proteinGroups.to_csv('Processing_Foldchange.csv')